In [1]:
import pyspark.sql.functions as f
import pyspark.sql.types as t
import json
from pyspark.sql.functions import from_json, col

In [2]:
df = spark.read.json('logs')

The following line gets the schema of every *.message of every line of the dataframe, and creates the schema to be able to read it as something complex, a.k.a. something not very well structured

In [3]:
json_schema = spark.read.json(df.rdd.map(lambda row: row.message)).schema

In [4]:
df2 = df.withColumn('message_json', from_json(f.col('message'), json_schema)).select(f.col('message_json.*'))

21/11/15 20:09:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df2.printSchema()

root
 |-- AA: boolean (nullable = true)
 |-- RA: boolean (nullable = true)
 |-- RD: boolean (nullable = true)
 |-- TC: boolean (nullable = true)
 |-- TTLs: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Z: long (nullable = true)
 |-- _corrupt_record: string (nullable = true)
 |-- acks: long (nullable = true)
 |-- actions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- active_dns_requests: long (nullable = true)
 |-- active_files: long (nullable = true)
 |-- active_icmp_conns: long (nullable = true)
 |-- active_tcp_conns: long (nullable = true)
 |-- active_timers: long (nullable = true)
 |-- active_udp_conns: long (nullable = true)
 |-- addl: string (nullable = true)
 |-- analyzers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- answers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- assigned_addr: string (nullable = true)
 |-- auth_attempts: long (nullable = true)
 |-

In [6]:
df2.count()

1000560

## HTTP Analysis

In [7]:
df_http = df2\
.filter(f.col('method').isNotNull())\
.select(
        "host", "`id.orig_h`", "`id.orig_p`", "`id.resp_h`", "`id.resp_p`", "method", "request_body_len",
        "response_body_len", "tags", "trans_depth", "ts", "uid", "uri", "user_agent"
)

In [8]:
df_http_response = df2\
.filter((f.col('`id.resp_p`') == '80') & (f.col('status_code').isNotNull()))\
.select (
        "`id.orig_h`", "`id.orig_p`", "`id.resp_h`", "`id.resp_p`", "request_body_len", "response_body_len",
        "status_code", "status_msg", "tags", "trans_depth", "ts", "uid", "version"
)

In [9]:
df_http.show(2, False)

+-------------------+------------+---------+--------------+---------+------+----------------+-----------------+----+-----------+-------------------+------------------+------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+
|host               |id.orig_h   |id.orig_p|id.resp_h     |id.resp_p|method|request_body_len|response_body_len|tags|trans_depth|ts                 |uid               |uri                                                                                                                     |user_agent                                     |
+-------------------+------------+---------+--------------+---------+------+----------------+-----------------+----+-----------+-------------------+------------------+------------------------------------------------------------------------------------------------------------------------+--------------------------------------

In [10]:
df_http_response.show(2, False)

+------------+---------+---------------+---------+----------------+-----------------+-----------+-----------------+----+-----------+-------------------+------------------+-------+
|id.orig_h   |id.orig_p|id.resp_h      |id.resp_p|request_body_len|response_body_len|status_code|status_msg       |tags|trans_depth|ts                 |uid               |version|
+------------+---------+---------------+---------+----------------+-----------------+-----------+-----------------+----+-----------+-------------------+------------------+-------+
|172.16.0.130|58468    |139.180.140.238|80       |0               |0                |301        |Moved Permanently|[]  |1          |1.635631196132812E9|CkArij2hV690t2Ddcl|1.1    |
|172.16.0.130|48308    |185.119.173.142|80       |0               |0                |301        |Moved Permanently|[]  |1          |1.635631199858157E9|CMhjxP0T6S3A34saf |1.1    |
+------------+---------+---------------+---------+----------------+-----------------+-----------+---

In [11]:
df_http_response.groupBy('status_code').count().sort(f.col('count').desc()).show(20, False)

+-----------+-----+
|status_code|count|
+-----------+-----+
|301        |50891|
|200        |14046|
|302        |6486 |
|403        |1915 |
|404        |689  |
|405        |458  |
|308        |391  |
|500        |325  |
|400        |266  |
|307        |252  |
|503        |189  |
|406        |181  |
|303        |111  |
|429        |89   |
|520        |58   |
|401        |34   |
|502        |29   |
|410        |18   |
|521        |9    |
|504        |9    |
+-----------+-----+
only showing top 20 rows



## DNS Analysis

In [12]:
df_dns = df2\
.filter(f.col('query').isNotNull())\
.select(
        "AA", "RA", "RD", "TC", "TTLs", "Z", "answers", "`id.orig_h`", "`id.orig_p`", "`id.resp_h`", "`id.resp_p`",
        "proto", "query", "rcode", "rcode_name", "rejected", "trans_id", "ts", "uid"
)

In [13]:
df_dns.filter(f.col('query').rlike('.*pudim.*')).show()

+-----+-----+----+-----+-----+---+---------------+------------+---------+----------+---------+-----+------------+-----+----------+--------+--------+-------------------+------------------+
|   AA|   RA|  RD|   TC| TTLs|  Z|        answers|   id.orig_h|id.orig_p| id.resp_h|id.resp_p|proto|       query|rcode|rcode_name|rejected|trans_id|                 ts|               uid|
+-----+-----+----+-----+-----+---+---------------+------------+---------+----------+---------+-----+------------+-----+----------+--------+--------+-------------------+------------------+
|false|false|true|false| null|  0|           null|172.16.0.134|    35484|172.16.0.2|       53|  udp|pudim.com.br|    0|   NOERROR|   false|   59958|1.635631685424454E9|CeHptQ1GNHV5c6340h|
|false| true|true|false|[5.0]|  0|[54.207.20.104]|172.16.0.134|    35484|172.16.0.2|       53|  udp|pudim.com.br|    0|   NOERROR|   false|   60720|1.635631685424452E9|CeHptQ1GNHV5c6340h|
|false|false|true|false| null|  0|           null|172.16.0.1

In [14]:
df_dns.groupBy('query').count().sort(f.col('count').desc()).show(20, False)

+--------------------------------+-----+
|query                           |count|
+--------------------------------+-----+
|www.hugedomains.com             |126  |
|registry-1.docker.io            |102  |
|www.google.com                  |90   |
|survey-smiles.com               |86   |
|www.facebook.com                |56   |
|pages.ebay.com                  |52   |
|www.tribpub.com                 |50   |
|www.ebay.com                    |48   |
|auth.docker.io                  |46   |
|mirrors.rockylinux.org          |46   |
|accounts.google.com             |46   |
|www.noripolice.com              |38   |
|secure.jbs.elsevierhealth.com   |34   |
|gcr.io                          |34   |
|production.cloudflare.docker.com|30   |
|www.microsoft.com               |30   |
|www.gov.uk                      |28   |
|dan.com                         |28   |
|mirror.uepg.br                  |24   |
|stopnote.vhostgo.com            |24   |
+--------------------------------+-----+
only showing top

In [15]:
df_dns.show(2, False)

+-----+----+-----+-----+----------+---+---------------------------------------+------------+---------+----------+---------+-----+------------------------+-----+----------+--------+--------+-------------------+------------------+
|AA   |RA  |RD   |TC   |TTLs      |Z  |answers                                |id.orig_h   |id.orig_p|id.resp_h |id.resp_p|proto|query                   |rcode|rcode_name|rejected|trans_id|ts                 |uid               |
+-----+----+-----+-----+----------+---+---------------------------------------+------------+---------+----------+---------+-----+------------------------+-----+----------+--------+--------+-------------------+------------------+
|false|true|false|false|[5.0, 5.0]|0  |[cherokeeheritage.org, 192.124.249.157]|172.16.0.130|55194    |172.16.0.2|53       |udp  |www.cherokeeheritage.org|0    |NOERROR   |false   |18053   |1.635631189704933E9|CBc1z02vdwFTL92Ori|
|false|true|false|false|[5.0]     |0  |[cherokeeheritage.org]                 |172.1

## SSL Analysis

In [16]:
df_ssl = df2\
.filter(f.col('cipher').isNotNull())\
.select(
        "cipher", "curve", "established", "`id.orig_h`", "`id.orig_p`", "`id.resp_h`", "`id.resp_p`", "resumed",
        "ts", "uid", "version"
)

In [17]:
df_ssl.show(2, False)

+-------------------------------------+---------+-----------+------------+---------+---------------+---------+-------+-------------------+------------------+-------+
|cipher                               |curve    |established|id.orig_h   |id.orig_p|id.resp_h      |id.resp_p|resumed|ts                 |uid               |version|
+-------------------------------------+---------+-----------+------------+---------+---------------+---------+-------+-------------------+------------------+-------+
|TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384|secp256r1|false      |172.16.0.130|33122    |185.119.173.142|443      |false  |1.635631200276187E9|CIuDS2395WzcrX3AA2|TLSv12 |
|TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384|secp256r1|false      |172.16.0.130|33122    |185.119.173.142|443      |false  |1.635631200276187E9|CIuDS2395WzcrX3AA2|TLSv12 |
+-------------------------------------+---------+-----------+------------+---------+---------------+---------+-------+-------------------+------------------+-------+
only

In [18]:
df_ssl.groupBy('version').count().sort(f.col('count').desc()).show(20, False)

+-------+-----+
|version|count|
+-------+-----+
|TLSv13 |12045|
|TLSv12 |6003 |
|TLSv10 |1    |
+-------+-----+

